In [3]:
# [1] 설치
!pip install transformers torch scikit-learn --quiet

In [4]:
# [2] 데이터셋 만들기 (간단 예시, 확장 가능)
lyrics = [
    "행복했던 기억이 가득해",        # 긍정(1)
    "너와의 이별이 너무 아파",        # 부정(0)
    "사랑은 다시 찾아올 거야",       # 긍정(1)
    "외로운 밤 혼자 울고 있어",      # 부정(0)
    "햇살처럼 눈부신 하루야",        # 긍정(1)
    "추억만이 남아 나를 괴롭혀",     # 부정(0)
    "새로운 시작이 설레여",         # 긍정(1)
    "상처뿐인 기억들이 떠올라",      # 부정(0)
    "희망을 안고 달려가",            # 긍정(1)
    "어둠 속에 방황하는 나"          # 부정(0)
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1=긍정, 0=부정

# [3] 토크나이저/데이터셋 준비
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
import numpy as np

# MPS 디바이스 설정 (Apple Silicon GPU 사용시 오류 방지)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    # MPS에서 발생할 수 있는 문제 방지
    torch.mps.empty_cache()
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")

# 토큰화 함수
def preprocess(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=32)
    return {
        'input_ids': torch.tensor(encodings['input_ids']),
        'attention_mask': torch.tensor(encodings['attention_mask']),
        'labels': torch.tensor(labels)
    }

# train/val 나누기
X_train, X_val, y_train, y_val = train_test_split(lyrics, labels, test_size=0.2, random_state=42)

train_enc = preprocess(X_train, y_train)
val_enc = preprocess(X_val, y_val)

# [4] PyTorch Dataset 래퍼
class LyricsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.encodings.items()}
    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = LyricsDataset(train_enc)
val_dataset = LyricsDataset(val_enc)

# [5] BERT 분류 모델 불러오기 (2 클래스: 긍정/부정)
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base-v2022", num_labels=2)
model.to(device)

# [6] Trainer로 학습
training_args = TrainingArguments(
    output_dir='./result',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='no',
    load_best_model_at_end=False,
    logging_dir='./logs',
    seed=42,
    report_to='none', # Explicitly disable wandb reporting
    use_mps_device=False if device.type == "mps" else None,  # MPS 호환성을 위해 비활성화
    dataloader_pin_memory=False  # MPS에서 메모리 문제 방지
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

# [7] 평가 및 새 가사 예측
print("\n--- 검증 데이터 예측 결과 ---")
outputs = trainer.predict(val_dataset)
preds = np.argmax(outputs.predictions, axis=1)
print("실제 라벨:", y_val)
print("예측 라벨:", preds.tolist())

# 새로운 가사 감정 예측
def predict_sentiment(lyrics_list):
    enc = tokenizer(lyrics_list, return_tensors='pt', truncation=True, padding=True, max_length=32)
    enc = {k: v.to(device) for k, v in enc.items()}  # 디바이스로 이동
    with torch.no_grad():
        out = model(**enc)
        probs = torch.softmax(out.logits, dim=1)
        preds = torch.argmax(probs, dim=1)
    for lyric, pred, prob in zip(lyrics_list, preds.cpu(), probs.cpu()):
        print(f'"{lyric}" => {"긍정" if pred==1 else "부정"} (긍정 확률: {prob[1]:.2f})')

print("\n--- 새로운 가사 감성 예측 ---")
test_lyrics = [
    "이별의 슬픔이 내 마음을 적셔",
    "희망찬 내일이 기다리고 있어",
    "내일은 더 행복할 거야",
    "눈물로 가득한 하루"
]
predict_sentiment(test_lyrics)

Using device: mps


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.684100,0.719197
2,0.644300,0.814952
3,0.564600,0.798058
4,0.579700,0.743914
5,0.550500,0.669864



--- 검증 데이터 예측 결과 ---


실제 라벨: [1, 0]
예측 라벨: [1, 0]

--- 새로운 가사 감성 예측 ---
"이별의 슬픔이 내 마음을 적셔" => 부정 (긍정 확률: 0.41)
"희망찬 내일이 기다리고 있어" => 부정 (긍정 확률: 0.47)
"내일은 더 행복할 거야" => 긍정 (긍정 확률: 0.61)
"눈물로 가득한 하루" => 부정 (긍정 확률: 0.46)
